In [64]:
# | default_exp utils.process

In [65]:
# | exporti
from dataclasses import dataclass, field
from typing import Any, Callable, List
from functools import wraps

import httpx

import domolibrary_extensions.client as dec

In [66]:
# | hide
from nbdev.showdoc import show_doc
import pandas as pd
import asyncio

# Process Functions

To simplify implementing work chains, a process function should issue API requests and capture actions taken including errors encountered. <br>

The function can be identified by and wrapped in a `process_function` wrapper class to ensure appropriate inputs and exports are sent to the class. <br>

A process function will always receive a `ProcessLog` and return a `ResponseProcess` class

In [67]:
# | exports


@dataclass
class ProcessMessage:
    stage: str # description of the stage of a process
    message: str = "init" # outcome
    is_success: bool = False

    """class for logging a stage of a process"""

    def to_json(self):
        return self.__dict__

@dataclass
class ProcessResponse:
    function_name: str
    id: str # identify a set of log entries
    
    message: List[ProcessMessage] = field(default_factory=lambda: []) # capture intermediate steps of the process_function
    response: Any = field(repr=False, default=None) # final object to return from the process_function
    is_success: bool = False

    """Response class for handling logging of a process function.
    Accumulates messages as the process unfolds
    """

    def to_json(self) -> List[dict]:
        columns = ['function_name', 'stage', 'id', 'is_success', 'message', 'response']
        s= [{**self.__dict__, **msg.to_json()} for msg in self.message]

        return [{col: obj[col] for col in columns}  for obj in s]
    
    

    def add_message(self, message: ProcessMessage):
        self.message.append(message)
    
    def __eq__(self, other):
        if not isinstance(other,ProcessResponse ):
            return False
        
        return self.id == other.id

@dataclass
class ProcessLogs:
    """process logs are the complete logs of an entire process or script"""

    logs: List[ProcessResponse] = field(default_factory=lambda: []) 

    def add_response(self, res: ProcessResponse):
        if res not in self.logs:
            self.logs.append(res)

        return self.logs
    
    def to_json(self) -> List[dict]:
        return [message for log in self.logs for message in log.to_json()]


In [68]:
#| exports
class ProcessFunction_Error(Exception):
    """base class for capturing errors within a process function"""

    def __init__(self, process: ProcessResponse, message: ProcessMessage, location=None):

        e = f"💀 | {process.function_name } | {process.id} | {message.stage} - {message.message}"

        if location:
            e = f"{e} | in {location}"

        super().__init__(e)

In [69]:
#| export

class ProcessFunction_ResponseTypeError(TypeError):
    """a function wrapped in `process_function` must return ResponseProcess class"""
    def __init__(self, result):
        super().__init__(
            f"Expected function to return an instance of ResponseProcess got {type(result)} instead.  Refactor function to return ResponseGetData class"
        )

def process_function_sync(func: Callable[..., Any]) -> Callable[..., Any]:
    """
    Decorator for synchronous route functions to ensure they receive certain arguments.
    If these arguments are not provided, default values are used.

    Args:
        func (Callable[..., Any]): The function to decorate.

    Returns:
        Callable[..., Any]: The decorated function.

    The decorated function takes the following arguments:
        *args (Any): Positional arguments for the decorated function.
        auth : dec.Auth
        log: Logs
        debug_api (bool, optional): Whether to debug the API. Defaults to False.
        session (httpx.AsyncClient, optional): The HTTPX client session. Defaults to None.
        **kwargs (Any): Additional keyword arguments for the decorated function.
    
    The decorated function must return ResponseProcess
    """

    @wraps(func)
    def wrapper(
        *args: Any,
        auth: dec.Auth,
        session: httpx.AsyncClient,
        debug_api: bool,
        logs: ProcessLogs,
        **kwargs: Any,
    ) -> Any:
        result = func(
            *args,
            auth=auth,
            debug_api=debug_api,
            session=session,
            logs=logs,
            **kwargs,
        )

        if not isinstance(result, ProcessResponse):
            raise ProcessFunction_ResponseTypeError(result)

        return result

    return wrapper

def process_function(func: Callable[..., Any]) -> Callable[..., Any]:
    """
    Decorator for async route functions to ensure they receive certain arguments.
    If these arguments are not provided, default values are used.

    Args:
        func (Callable[..., Any]): The function to decorate.

    Returns:
        Callable[..., Any]: The decorated function.

    The decorated function takes the following arguments:
        *args (Any): Positional arguments for the decorated function
        auth : dec.Auth
        log: Logs
        debug_api (bool, optional): Whether to debug the API. Defaults to False.
        session (httpx.AsyncClient, optional): The HTTPX client session. Defaults to None.
        **kwargs (Any): Additional keyword arguments for the decorated function.
    
    The decorated function must return ResponseProcess
    """

    @wraps(func)
    async def wrapper(
        *args: Any,
        auth: dec.Auth,
        session: httpx.AsyncClient,
        debug_api: bool,
        logs: ProcessLogs,
        **kwargs: Any,
    ) -> Any:
        result = await func(
            *args,
            auth=auth,
            debug_api=debug_api,
            session=session,
            logs=logs,
            **kwargs,
        )

        if not isinstance(result, ProcessResponse):
            raise ProcessFunction_ResponseTypeError(result)

        return result

    return wrapper

In [70]:
show_doc(process_function)

---

### process_function

>      process_function (func:Callable[...,Any])

Decorator for async route functions to ensure they receive certain arguments.
If these arguments are not provided, default values are used.

Args:
    func (Callable[..., Any]): The function to decorate.

Returns:
    Callable[..., Any]: The decorated function.

The decorated function takes the following arguments:
    *args (Any): Positional arguments for the decorated function
    auth : dec.Auth
    log: Logs
    debug_api (bool, optional): Whether to debug the API. Defaults to False.
    session (httpx.AsyncClient, optional): The HTTPX client session. Defaults to None.
    **kwargs (Any): Additional keyword arguments for the decorated function.

The decorated function must return ResponseProcess

In [71]:
show_doc(process_function_sync)

---

### process_function_sync

>      process_function_sync (func:Callable[...,Any])

Decorator for synchronous route functions to ensure they receive certain arguments.
If these arguments are not provided, default values are used.

Args:
    func (Callable[..., Any]): The function to decorate.

Returns:
    Callable[..., Any]: The decorated function.

The decorated function takes the following arguments:
    *args (Any): Positional arguments for the decorated function.
    auth : dec.Auth
    log: Logs
    debug_api (bool, optional): Whether to debug the API. Defaults to False.
    session (httpx.AsyncClient, optional): The HTTPX client session. Defaults to None.
    **kwargs (Any): Additional keyword arguments for the decorated function.

The decorated function must return ResponseProcess

In [72]:
auth = "abc"  # must use domolibrary_extensions.client.Auth class

logs = ProcessLogs()  # for logging the overall process


@process_function
async def process_user(
    auth,
    username,  # used to identify a set of log messages
    logs,  # a mutable object that accumulates logs
    # pass to dc.get_data
    session: httpx.AsyncClient = None,  # support asynchornous code execution
    debug_api: bool = False,  # pass to dc.get_data to facilitate debugging
):
    #### PROCESS INITIALIZATION ####
    # process should begin with base implementation of the response object and then accumulate logs
    res = ProcessResponse(
        function_name="process_user",
        id=username,  # identifies a set of related log entries
    )
    logs.add_response(res)

    #### FIRST STEP ####
    message = ProcessMessage(
        stage="get user"
    )  # inititalize a message for each step of the process
    try:
        final_output = "stuff"
        message.message = f"user retrieved"
        message.is_success = True

    except Exception as e:
        message.message = e

    # add message to the ProcessResponse logging
    res.add_message(message)

    # secod step
    message = ProcessMessage(
        stage="reset_password"
    )  # inititalize a message for each step of the process
    try:
        final_output += "| more stuff"
        message.message = "password updated"
        message.is_success = True

    except Exception as e:
        message.message = e

    # add message to the ProcessResponse logging
    res.add_message(message)

    # done // update ProcessResponse and return
    res.response = final_output
    res.is_success = True

    return res

In [73]:
# each executio of the process fuctio can be represened as a dataframe
log = ProcessLogs()

res = await process_user(
    username="baz", auth=auth,
    logs=logs,
    session=httpx.AsyncClient(),
    debug_api=False
)

res

pd.DataFrame(res.to_json())

,function_name,stage,id,is_success,message,response
0,process_user,get user,baz,True,user retrieved,stuff| more stuff
1,process_user,reset_password,baz,True,password updated,stuff| more stuff


In [74]:
log = ProcessLogs()

users_to_process = ["foo", "baz", "shoe"]


await asyncio.gather(
    *[
        process_user(
            username=user,
            auth=auth,
            logs=logs,
            session=httpx.AsyncClient(),
            debug_api=False,
        )
        for user in users_to_process
    ]
)

pd.DataFrame(logs.to_json())

,function_name,stage,id,is_success,message,response
0,process_user,get user,baz,True,user retrieved,stuff| more stuff
1,process_user,reset_password,baz,True,password updated,stuff| more stuff
2,process_user,get user,foo,True,user retrieved,stuff| more stuff
3,process_user,reset_password,foo,True,password updated,stuff| more stuff
4,process_user,get user,shoe,True,user retrieved,stuff| more stuff
5,process_user,reset_password,shoe,True,password updated,stuff| more stuff


In [76]:
# | hide
import nbdev

nbdev.nbdev_export("./process.ipynb")